In [5]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### 1. Train model

#### 1.1. Describe data and split dataset

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [3]:
df = describe_dataset("./train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 15372 
Number of columns: 37

Labels: 
C    8238
L    7134
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [6]:
# Extract features and class
X = df.drop("label", axis=1) # features
y = df["label"]

In [8]:
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_test.head(5)

1922     L
6789     L
8933     L
1985     L
14857    C
Name: label, dtype: object

#### 1.2. Train model using Scikit-learn

In [10]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []

for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=["C", "L"])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=["C", "L"])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=["C", "L"])
    cm = confusion_matrix(y_test, model_results, labels=["C", "L"])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))

# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,RF,"[0.999, 0.998]",0.998699,"[0.998, 0.999]","[0.999, 0.999]","[[1676, 3], [1, 1395]]"
1,KNN,"[0.997, 0.999]",0.998049,"[0.999, 0.996]","[0.998, 0.998]","[[1678, 1], [5, 1391]]"
2,SVC,"[0.997, 0.995]",0.996098,"[0.996, 0.996]","[0.996, 0.996]","[[1672, 7], [5, 1391]]"
3,DTC,"[0.997, 0.991]",0.994472,"[0.993, 0.996]","[0.995, 0.994]","[[1667, 12], [5, 1391]]"
4,LR,"[0.986, 0.975]",0.980813,"[0.979, 0.983]","[0.982, 0.979]","[[1644, 35], [24, 1372]]"
5,SGDC,"[0.984, 0.975]",0.979837,"[0.979, 0.981]","[0.981, 0.978]","[[1644, 35], [27, 1369]]"
6,NB,"[0.927, 0.842]",0.884878,"[0.857, 0.918]","[0.89, 0.879]","[[1439, 240], [114, 1282]]"


#### 1.3. Evaluation with Test set

In [11]:
test_df = describe_dataset("./test.csv")
test_df = test_df.sample(frac=1).reset_index(drop=True)

test_x = test_df.drop("label", axis=1)
test_y = test_df["label"]

test_x = pd.DataFrame(sc.transform(test_x))

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 604 
Number of columns: 37

Labels: 
C    339
L    265
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [12]:
testset_final_results = []

for name, model in models.items():
    # Evaluate model
    model_results = model.predict(test_x)

    p_score = precision_score(test_y, model_results, average=None, labels=["C", "L"])
    a_score = accuracy_score(test_y, model_results)
    r_score = recall_score(test_y, model_results, average=None, labels=["C", "L"])
    f1_score_result = f1_score(test_y, model_results, average=None, labels=["C", "L"])
    cm = confusion_matrix(test_y, model_results, labels=["C", "L"])
    testset_final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm ))


testset_final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(testset_final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,KNN,"[0.955, 0.996]",0.971854,"[0.997, 0.94]","[0.975, 0.967]","[[338, 1], [16, 249]]"
1,RF,"[0.91, 0.975]",0.935430,"[0.982, 0.875]","[0.945, 0.922]","[[333, 6], [33, 232]]"
2,SVC,"[0.957, 0.903]",0.932119,"[0.92, 0.947]","[0.938, 0.924]","[[312, 27], [14, 251]]"
3,DTC,"[0.732, 0.979]",0.791391,"[0.991, 0.536]","[0.842, 0.693]","[[336, 3], [123, 142]]"
4,LR,"[0.723, 0.862]",0.761589,"[0.932, 0.543]","[0.814, 0.667]","[[316, 23], [121, 144]]"
5,SGDC,"[0.708, 0.769]",0.726821,"[0.873, 0.54]","[0.782, 0.634]","[[296, 43], [122, 143]]"
6,NB,"[0.595, 1.0]",0.617550,"[1.0, 0.128]","[0.746, 0.227]","[[339, 0], [231, 34]]"


#### 1.4. Dump model pickle

In [13]:
with open("./model/all_sklearn.pkl", "wb") as f:
    pickle.dump(models, f)

In [39]:
with open("./model/RF_model.pkl", "wb") as f:
    pickle.dump(models["RF"], f)

In [40]:
with open("./model/KNN_model.pkl", "wb") as f:
    pickle.dump(models["KNN"], f)

In [43]:
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)